In [ ]:

import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.model_selection import train_test_split


data_path = os.path.abspath(os.path.join("1","..", "..", "..", "data", "external",'spotify-2' ,"train.csv"))
test_data_path = os.path.abspath(os.path.join("1","..", "..", "..", "data", "external",'spotify-2' ,"test.csv"))
val_data_path = os.path.abspath(os.path.join("1","..", "..", "..", "data", "external",'spotify-2' ,"validate.csv"))

# using pandas to read the csv file into our dataframe named df
df = pd.read_csv(data_path)
df_test = pd.read_csv(test_data_path)
df_val = pd.read_csv(val_data_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import os
import sys

sys.path.append(os.path.abspath('../../models/knn'))
from knn import KNNClassifier
from knn import Metrics

def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype.name != 'object':
            df[col].fillna(df[col].mean(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    return df

df = impute_missing_values(df)
df_test = impute_missing_values(df_test)


def custom_label_encoder(train_series, test_series):
    unique_vals = train_series.unique()
    val_to_int = {val: idx for idx, val in enumerate(unique_vals)}
    train_encoded = train_series.map(val_to_int)
    test_encoded = test_series.map(val_to_int)
    return train_encoded, test_encoded, val_to_int

def label_encode_columns(df_train, df_test, columns):
    label_encoders = {}
    for col in columns:
        df_train[col], df_test[col], encoder = custom_label_encoder(df_train[col], df_test[col])
        label_encoders[col] = encoder
    return df_train, df_test, label_encoders

string_columns = ['artists', 'album_name', 'explicit']

df, df_test, label_encoders = label_encode_columns(df, df_test, string_columns)


df.drop(columns=['track_id', 'track_name','Unnamed: 0'], inplace=True)
df_test.drop(columns=['track_id', 'track_name','Unnamed: 0'], inplace=True)


df['track_genre'] = df['track_genre'].astype('category').cat.codes
df_test['track_genre'] = df_test['track_genre'].astype('category').cat.codes

df = impute_missing_values(df)
df_test = impute_missing_values(df_test)



X_train = df.drop(columns=['track_genre'])
X_test = df_test.drop(columns=['track_genre'])
y_train = df['track_genre']
y_test = df_test['track_genre']

def standardize(X_train, X_test, categorical_columns):
    # Standardize only numerical columns
    num_columns = X_train.columns.difference(categorical_columns)
    
    mean = X_train[num_columns].mean(axis=0)
    std = X_train[num_columns].std(axis=0)
    
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[num_columns] = (X_train[num_columns] - mean) / std
    X_test_scaled[num_columns] = (X_test[num_columns] - mean) / std
    
    return X_train_scaled, X_test_scaled

categorical_columns = string_columns

X_train_scaled, X_test_scaled = standardize(X_train, X_test, categorical_columns)

# X_train_scaled.to_csv('train_cleaned.csv', index=False)
# knn = KNNClassifier(k=5, distance_metric='euclidean')
# knn.fit(X_train_scaled.values, y_train.values)

# # Predict on test data
# y_pred = knn.predict(X_test_scaled.values)

knn = knn(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

# Calculate and print metrics
metrics = Metrics()
accuracy = metrics.accuracy(y_test.values, y_pred)
precision = metrics.precision(y_test.values, y_pred)
recall = metrics.recall(y_test.values, y_pred)
f1 = metrics.f1_score(y_test.values, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
import os
import sys

sys.path.append(os.path.abspath('../../models/knn'))
from knn import KNNClassifier

def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype.name != 'object':
            df[col].fillna(df[col].mean(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    return df

def custom_label_encoder(train_series, test_series, val_series=None):
    unique_vals = pd.concat([train_series, test_series, val_series]) if val_series is not None else pd.concat([train_series, test_series])
    unique_vals = unique_vals.unique()
    val_to_int = {val: idx for idx, val in enumerate(unique_vals)}
    train_encoded = train_series.map(val_to_int)
    test_encoded = test_series.map(val_to_int)
    val_encoded = val_series.map(val_to_int) if val_series is not None else None
    return train_encoded, test_encoded, val_encoded, val_to_int

def label_encode_columns(df_train, df_test, df_val, columns):
    label_encoders = {}
    for col in columns:
        df_train[col], df_test[col], df_val[col], encoder = custom_label_encoder(df_train[col], df_test[col], df_val[col])
        label_encoders[col] = encoder
    return df_train, df_test, df_val, label_encoders

data_path = os.path.abspath(os.path.join("1", "..", "..", "..", "data", "external", 'spotify-2', "train.csv"))
test_data_path = os.path.abspath(os.path.join("1", "..", "..", "..", "data", "external", 'spotify-2', "test.csv"))
val_data_path = os.path.abspath(os.path.join("1", "..", "..", "..", "data", "external", 'spotify-2', "validate.csv"))


df = pd.read_csv(data_path)
df_test = pd.read_csv(test_data_path)
df_val = pd.read_csv(val_data_path)

df = impute_missing_values(df)
df_test = impute_missing_values(df_test)
df_val = impute_missing_values(df_val)

string_columns = ['artists', 'album_name', 'explicit']
df, df_test, df_val, label_encoders = label_encode_columns(df, df_test, df_val, string_columns)

genre_encoder = pd.concat([df['track_genre'], df_test['track_genre'], df_val['track_genre']]).astype('category').cat.codes
df['track_genre'] = genre_encoder[:len(df)]
df_test['track_genre'] = genre_encoder[len(df):len(df) + len(df_test)]
df_val['track_genre'] = genre_encoder[len(df) + len(df_test):]

columns_to_drop = ['track_id', 'track_name', 'Unnamed: 0']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_test.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_val.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Define features and target
X_train = df.drop(columns=['track_genre'])
X_test = df_test.drop(columns=['track_genre'])
X_val = df_val.drop(columns=['track_genre'])
y_train = df['track_genre']
y_test = df_test['track_genre']
y_val = df_val['track_genre']

def standardize(X_train, X_test, X_val, categorical_columns):
    num_columns = X_train.columns.difference(categorical_columns)
    
    mean = X_train[num_columns].mean(axis=0)
    std = X_train[num_columns].std(axis=0)
    
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_val_scaled = X_val.copy()
    
    X_train_scaled[num_columns] = (X_train[num_columns] - mean) / std
    X_test_scaled[num_columns] = (X_test[num_columns] - mean) / std
    X_val_scaled[num_columns] = (X_val[num_columns] - mean) / std
    
    return X_train_scaled, X_test_scaled, X_val_scaled

categorical_columns = string_columns
X_train_scaled, X_test_scaled, X_val_scaled = standardize(X_train, X_test, X_val, categorical_columns)


def evaluate_knn(k, distance_metric):
    knn = KNeighborsClassifier(n_neighbors=k, metric=distance_metric)
    knn.fit(X_train_scaled.values, y_train.values)
    y_pred_val = knn.predict(X_val_scaled.values)
    
    accuracy = np.mean(y_pred_val == y_val.values)
    return accuracy

k_values = [1, 3, 5, 7, 9]
distance_metrics = ['euclidean', 'manhattan']

results = []
for k, distance_metric in product(k_values, distance_metrics):
    accuracy = evaluate_knn(k, distance_metric)
    results.append((k, distance_metric, accuracy))

results.sort(key=lambda x: x[2], reverse=True)

print("Top 10 (k, distance_metric) pairs:")
for i, (k, distance_metric, accuracy) in enumerate(results[:10]):
    print(f"{i+1}. k={k}, distance_metric={distance_metric}, accuracy={accuracy:.4f}")

selected_metric = 'euclidean' 
ks = [k for k, metric, _ in results if metric == selected_metric]
accuracies = [accuracy for k, metric, accuracy in results if metric == selected_metric]

plt.plot(ks, accuracies, marker='o')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs k (distance_metric={selected_metric})')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from itertools import product



sys.path.append(os.path.abspath('../../models/knn'))
from knn import KNNClassifier  # Custom KNN classifier implementation

# Load data
data_path = os.path.abspath(os.path.join("1", "..", "..", "..", "data", "external", 'spotify-2', "train.csv"))
df = pd.read_csv(data_path)

# Function to impute missing values
def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype.name != 'object':
            df[col].fillna(df[col].mean(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    return df

# Impute missing values in the dataset
df = impute_missing_values(df)

# Label encode categorical columns
def custom_label_encoder(series):
    unique_vals = series.unique()
    val_to_int = {val: idx for idx, val in enumerate(unique_vals)}
    encoded = series.map(val_to_int)
    return encoded, val_to_int

def label_encode_columns(df, columns):
    label_encoders = {}
    for col in columns:
        df[col], encoder = custom_label_encoder(df[col])
        label_encoders[col] = encoder
    return df, label_encoders

string_columns = ['artists', 'album_name', 'explicit']
df, label_encoders = label_encode_columns(df, string_columns)

# Drop unnecessary columns
df.drop(columns=['track_id', 'track_name', 'Unnamed: 0'], inplace=True, errors='ignore')

# Encode target variable
df['track_genre'], genre_encoder = custom_label_encoder(df['track_genre'])

# Split the dataset into train, test, and validation sets
def train_test_val_split(df, train_size=0.7, test_size=0.15):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(df))
    train_end = int(train_size * len(df))
    test_end = int(test_size * len(df)) + train_end
    
    train_indices = shuffled_indices[:train_end]
    test_indices = shuffled_indices[train_end:test_end]
    val_indices = shuffled_indices[test_end:]
    
    return df.iloc[train_indices], df.iloc[test_indices], df.iloc[val_indices]

df_train, df_test, df_val = train_test_val_split(df)

# Define features and target
X_train = df_train.drop(columns=['track_genre'])
X_test = df_test.drop(columns=['track_genre'])
X_val = df_val.drop(columns=['track_genre'])

y_train = df_train['track_genre']
y_test = df_test['track_genre']
y_val = df_val['track_genre']

# Standardize numerical features
def standardize(X_train, X_test, X_val, categorical_columns):
    num_columns = X_train.columns.difference(categorical_columns)
    
    mean = X_train[num_columns].mean(axis=0)
    std = X_train[num_columns].std(axis=0)
    
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_val_scaled = X_val.copy()
    
    X_train_scaled[num_columns] = (X_train[num_columns] - mean) / std
    X_test_scaled[num_columns] = (X_test[num_columns] - mean) / std
    X_val_scaled[num_columns] = (X_val[num_columns] - mean) / std
    
    return X_train_scaled, X_test_scaled, X_val_scaled

X_train_scaled, X_test_scaled, X_val_scaled = standardize(X_train, X_test, X_val, string_columns)

def evaluate_knn(k, distance_metric):
    knn = KNeighborsClassifier(n_neighbors=k, metric=distance_metric)
    knn.fit(X_train_scaled.values, y_train.values)
    y_pred_val = knn.predict(X_val_scaled.values)
    
    accuracy = np.mean(y_pred_val == y_val.values)
    return accuracy

k_values = [1, 3, 5, 7, 9]
distance_metrics = ['euclidean', 'manhattan']

results = []
for k, distance_metric in product(k_values, distance_metrics):
    accuracy = evaluate_knn(k, distance_metric)
    results.append((k, distance_metric, accuracy))

results.sort(key=lambda x: x[2], reverse=True)

print("Top 10 (k, distance_metric) pairs:")
for i, (k, distance_metric, accuracy) in enumerate(results[:10]):
    print(f"{i+1}. k={k}, distance_metric={distance_metric}, accuracy={accuracy:.4f}")

selected_metric = 'euclidean' 
ks = [k for k, metric, _ in results if metric == selected_metric]
accuracies = [accuracy for k, metric, accuracy in results if metric == selected_metric]

plt.plot(ks, accuracies, marker='o')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs k (distance_metric={selected_metric})')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as knn
import os
import sys

sys.path.append(os.path.abspath('../../models/knn'))
from knn import KNNClassifier
from knn import Metrics

def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype.name != 'object':
            df[col].fillna(df[col].mean(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    return df

df = impute_missing_values(df)
df_test = impute_missing_values(df_test)


def custom_label_encoder(train_series, test_series):
    unique_vals = train_series.unique()
    val_to_int = {val: idx for idx, val in enumerate(unique_vals)}
    train_encoded = train_series.map(val_to_int)
    test_encoded = test_series.map(val_to_int)
    return train_encoded, test_encoded, val_to_int

def label_encode_columns(df_train, df_test, columns):
    label_encoders = {}
    for col in columns:
        df_train[col], df_test[col], encoder = custom_label_encoder(df_train[col], df_test[col])
        label_encoders[col] = encoder
    return df_train, df_test, label_encoders

string_columns = ['artists', 'album_name', 'explicit']

df, df_test, label_encoders = label_encode_columns(df, df_test, string_columns)


df.drop(columns=['track_id', 'track_name','Unnamed: 0'], inplace=True)
df_test.drop(columns=['track_id', 'track_name','Unnamed: 0'], inplace=True)


df['track_genre'] = df['track_genre'].astype('category').cat.codes
df_test['track_genre'] = df_test['track_genre'].astype('category').cat.codes

df = impute_missing_values(df)
df_test = impute_missing_values(df_test)



X_train = df.drop(columns=['track_genre'])
X_test = df_test.drop(columns=['track_genre'])
y_train = df['track_genre']
y_test = df_test['track_genre']

def standardize(X_train, X_test, categorical_columns):
    # Standardize only numerical columns
    num_columns = X_train.columns.difference(categorical_columns)
    
    mean = X_train[num_columns].mean(axis=0)
    std = X_train[num_columns].std(axis=0)
    
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[num_columns] = (X_train[num_columns] - mean) / std
    X_test_scaled[num_columns] = (X_test[num_columns] - mean) / std
    
    return X_train_scaled, X_test_scaled

categorical_columns = string_columns

X_train_scaled, X_test_scaled = standardize(X_train, X_test, categorical_columns)

X_train_scaled.to_csv('train_cleaned.csv', index=False)
# knn = KNNClassifier(k=5, distance_metric='euclidean')
# knn.fit(X_train_scaled.values, y_train.values)

# # Predict on test data
# y_pred = knn.predict(X_test_scaled.values)

knn = knn(n_neighbors=1)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

# Calculate and print metrics
metrics = Metrics()
accuracy = Metrics.accuracy(y_test.values, y_pred)
precision = Metrics.precision(y_test.values, y_pred)
recall = Metrics.recall(y_test.values, y_pred)
f1 = Metrics.f1_score(y_test.values, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")